Step 1: Setup and Load Data


In [8]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("TMDB Analysis") \
    .getOrCreate()

# Load the datasets
movies_df = spark.read.csv("dataset/tmdb_5000_movies.csv", header=True, inferSchema=True)
credits_df = spark.read.csv("dataset/tmdb_5000_credits.csv", header=True, inferSchema=True)

# Show schema and initial rows
movies_df.printSchema()
movies_df.show(5)
credits_df.printSchema()
credits_df.show(5)


root
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)

+---------+-------------+--------------------+------------+--------------------+-----------------+--------------------+--------------------+-------------

Step 2: Merge Dataframes


In [9]:
# Merge datasets on the movie ID
df = movies_df.join(credits_df, movies_df.id == credits_df.movie_id, 'inner').drop(credits_df.movie_id)

# Show merged dataframe
df.printSchema()
df.show(5)

# Save the initial merged dataframe to CSV
df.toPandas().to_csv('data/merged_data.csv', index=False)


root
 |-- budget: string (nullable = true)
 |-- genres: string (nullable = true)
 |-- homepage: string (nullable = true)
 |-- id: string (nullable = true)
 |-- keywords: string (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- status: string (nullable = true)
 |-- tagline: string (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: string (nullable = true)
 |-- vote_count: string (nullable = true)
 |-- title: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- crew: string (nullable = true)

+-------+------+--------+------+--------+---

Step 3: Data Cleaning and Preprocessing


In [10]:
from pyspark.sql.functions import col

# Convert necessary columns to appropriate data types
df = df.withColumn("budget", col("budget").cast("float")) \
       .withColumn("revenue", col("revenue").cast("float")) \
       .withColumn("runtime", col("runtime").cast("float")) \
       .withColumn("popularity", col("popularity").cast("float"))

# Handle missing values
df = df.dropna(subset=['budget', 'revenue', 'runtime', 'popularity'])

# Save cleaned data to CSV
df.toPandas().to_csv('data/cleaned_data.csv', index=False)


Step 4: Feature Engineering and Data Transformation


In [11]:
from pyspark.sql.functions import split, explode
from pyspark.ml.feature import StringIndexer, VectorAssembler

# Extract relevant features from JSON columns
# Assuming genres, cast, crew are JSON strings, explode them for feature extraction
# Simplifying by just taking the length of these lists as a feature
df = df.withColumn("genres_count", explode(split(col('genres'), ","))) \
       .withColumn("cast_count", explode(split(col('cast'), ","))) \
       .withColumn("crew_count", explode(split(col('crew'), ",")))

# Convert categorical columns to numerical values using StringIndexer
indexer = StringIndexer(inputCol="original_language", outputCol="original_language_index")
df = indexer.fit(df).transform(df)

# Assemble features into a single vector column for machine learning models
feature_columns = ['budget', 'popularity', 'runtime', 'original_language_index']
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df = assembler.transform(df)

# Show dataframe with features
df.select("features", "revenue").show(5)


+--------------------+---------+
|            features|  revenue|
+--------------------+---------+
|[0.0,0.2147040069...|      0.0|
|(4,[1],[0.0278009...|      0.0|
|[2000000.0,0.0319...|1672730.0|
|(4,[1],[0.0011859...|      0.0|
|[0.0,0.6900889873...|      0.0|
+--------------------+---------+
only showing top 5 rows



Step 5: Split Data into Training and Testing Sets


In [12]:
# Split the data into training and testing sets
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Save the split datasets to CSV
train_df.toPandas().to_csv('data/train_data.csv', index=False)
test_df.toPandas().to_csv('data/test_data.csv', index=False)


Step 6: Regression Analysis to Predict Movie Revenue


In [14]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

# Implement a Linear Regression model to predict movie revenue
lr = LinearRegression(featuresCol='features', labelCol='revenue')
lr_model = lr.fit(train_df)
lr_predictions = lr_model.transform(test_df)

# Evaluate the Linear Regression model
lr_evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='revenue', metricName='rmse')
rmse = lr_evaluator.evaluate(lr_predictions)

print(f"Root Mean Squared Error (RMSE) on test data = {rmse}")

# Save regression predictions to CSV
lr_predictions.toPandas().to_csv('data/lr_predictions.csv', index=False)


Root Mean Squared Error (RMSE) on test data = 1358.61597177138


Step 7: Classification Analysis to Predict Movie Success


In [16]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create success category based on revenue
df = df.withColumn('success', when(col('revenue') > 100000000, 1).otherwise(0))

# Split the data again as we added a new column
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)

# Train a RandomForest Classifier
rf = RandomForestClassifier(featuresCol='features', labelCol='success')
rf_model = rf.fit(train_df)
rf_predictions = rf_model.transform(test_df)

# Evaluate the RandomForest Classifier model
rf_evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='success', metricName='accuracy')
accuracy = rf_evaluator.evaluate(rf_predictions)

print(f"Accuracy on test data = {accuracy}")

# Save classification predictions to CSV
rf_predictions.toPandas().to_csv('data/rf_predictions.csv', index=False)


Accuracy on test data = 1.0


Step 8: Export Final Processed Data for Visualization in Tableau


In [17]:
# Save final dataframe to CSV for Tableau
df.toPandas().to_csv('data/final_data_for_tableau.csv', index=False)

# Stop the Spark session
spark.stop()
